In [2]:
import os, json, glob, shutil
from PIL import Image
from shapely import geometry       #conda install -c conda-forge shapely
import PIL.ImageDraw as ImageDraw
import numpy as np
from sklearn.model_selection import train_test_split
import pandas as pd
import cv2            #pip install opencv-python
from matplotlib import pyplot as plt
from tqdm import tqdm

In [3]:
def read_json_file(json_path):
    with open(json_path, encoding='utf-8-sig') as f:
        data = json.load(f)
    return data

In [4]:
root_dir = "datasets/"

# <set the path to json and image files>
json_file_path = root_dir + "json/*.json"
image_file_path = root_dir + 'image/*.tif'
save_path = os.path.basename(root_dir)
json_files = glob.glob(json_file_path) 
image_files  = glob.glob(image_file_path, recursive=True)

In [5]:
print("Read polygons...")
total_polygons = []
dataset = []
for i in range(len(image_files)):   
    image_name = os.path.basename(image_files[i])
    json_file = [ j for j in json_files if image_name in j]
    if len(json_file) == 0:
        dataset.append({"image-name":  image_files[i],'polygons':[],'points':[], 'shift':[]})
        continue
    json_file = json_file[0]
    json_data = read_json_file(json_file)
    each_image_data = []
    polygons = []
    points = []
    shifts = []
    each_image_data = {"image-name": image_files[i]}  
    for annotation in json_data['annotations']:
        polygons.append(geometry.Polygon(annotation['polygon.points']))
        points.append(annotation['polygon.points'])
        shifts.append(annotation['polygon.shift'])
        total_polygons.append(geometry.Polygon(annotation['polygon.points']))
    
    each_image_data['polygons'] = polygons
    each_image_data['points'] = points
    each_image_data['shifts'] = shifts
    dataset.append(each_image_data)

Read polygons...


In [7]:
#print(dataset[0])
cnt = 1
for data in tqdm(dataset):
    img_name = data['image-name']
    points = data['points']
    img = cv2.imread(img_name)
    #0신축 1소멸 2갱신
    for point in points:
        pts = np.array(point, np.int32).reshape((-1,1,2))
        cv2.polylines(img,[pts], True, (0,0,255), thickness = 3)
    cnt+=1
    save_name = save_path+"datasets/polygon/" + str(cnt)+ ".png"
    #print(save_name)
    cv2.imwrite(save_name, img)

100%|████████████████████████████████████████████████████████████████████████████████| 360/360 [01:01<00:00,  5.84it/s]
